# Absenteeism-Preprocessing

In [1]:
import pandas as pd
import numpy as np
import sys

### Step 1
Write a function that processes a single .txt file. It must:
* drop rows that do not contain a unique school identifier.
* drop rows that correspond to elementary/middle school education. We are focusing on high school data.

In [2]:
def read_text_file(textfilepath):
    """
    Input: textfilepath, a path to the text file to be generated to a datafrmae
    Output: Pandas DataFrame corresponding to input text file
    """
    df = pd.read_csv(textfilepath, sep="\t", encoding="ISO-8859-1")
    return df 

In [3]:
df2016 = read_text_file("./txt_files/absenteeism_txt_files/2016-17_ChronAbsenteeism.txt")
df2017 = read_text_file("./txt_files/absenteeism_txt_files/2017-18_ChronAbsenteeism.txt")
df2018 = read_text_file("./txt_files/absenteeism_txt_files/2018-19_ChronAbsenteeism.txt")
#df2020 = read_text_file("./txt_files/absenteeism_txt_files/2020-21_ChronAbsenteeism.txt")

In [4]:
def drop_rows(df):
    """
    Input: Pandas DataFrame
    Output: Pandas DataFrame with (a) rows with no school code and (b) rows corresponding to elementary/
            middle school education removed
    """
    rows_to_drop = []
    #drop rows that do not have unique school code
    for i, code in enumerate(df["SchoolCode"]):
        if pd.isnull(code) or code in [0,1]:
            rows_to_drop.append(i)
    #drop rows that correspond to elementary and middle school data
    for i, reporting_category in enumerate(df["ReportingCategory"]):
        if reporting_category in ["GRKN", "GRK", "GR13", "GR46", "GR78", "GRK8", "GR912", "GRUG"]:
            rows_to_drop.append(i)
    df = df.drop(rows_to_drop,axis=0)
    return df

In [5]:
df2016 = drop_rows(df2016)
df2017 = drop_rows(df2017)
df2018 = drop_rows(df2018)

### Step 2
We only want one row per school. As of now, the DataFrame has multiple rows per school to give metrics across different reporting groups.

To do this, we will construct of matrix where rows are individual schools and columns are 
["RB", "RI", "RA", "RF", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"]

These are each of the reporting categories.

In [6]:
def generate_column_index_mapping(columns):
    """
    Generates a mapping encoded as a dictionary where the keys are data columns (Absenteeism
    Rates for different demographic groups) and values are numeric values/
    Inputs:
    columns: list of data columns (i.e. ["RB", "RI", etc.])
    Outputs:
    mapping: dictionary where keys are inputs columns and values are numeric encodings.
             format: {"RB": 0, "RI": 1, etc}
    """
    mapping = {}
    for i,col in enumerate(columns):
        mapping[col] = i
    return mapping
mapping = generate_column_index_mapping(["RB", "RI", "RA", "RF", "RH", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"])

In [7]:
all_columns = ["AcademicYear", "AggregateLevel", "CountyCode", "DistrictCode", "SchoolCode", "CountyName",
          "DistrictName", "SchoolName", "CharterYN", "CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]
reporting_category_columns = ["CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]

In [8]:
def colapse_df(df):
    """
    The input dataframe has many rows of data for a single school. For each school, there is 
    a separate row for each Chronic Absenteeism reporting group (i.e. one for female students,
    one for white students, etc.) Using generate_column_index_mapping(columns), this function
    groups all the data for one school into a single row and adds the columns "CAR_RB", "CAR_RI",
    etc. to record the chronic absenteeism rates for each reporting category in a single row.
    Input:
    df: dataframe whose only preprocessing is done by drop_rows()
    Output:
    new_df: dataframe such that chronic absenteeism rates for each reporting group are added as
            columns, and each school only has 1 row of data.
    """
    #determine which rows are associated with each school
    school_codes = df["SchoolCode"].unique()
    associated_rows = {}
    for code in school_codes:
        associated_rows[code] = []
    for i in range(df.shape[0]):
        #df["SchoolCode"][df.index[i]] = school code at row i of df
        associated_rows[df["SchoolCode"][df.index[i]]].append(i)
        
    array = np.resize(np.array(all_columns), (1,len(all_columns))) #resize to stack rows with numpy
    for code in school_codes:
        rows = associated_rows[code] #get rows associated with code
        firstrow = df.iloc[rows[0]] #use first row to capture redundant data
        school_data = np.array([[firstrow["AcademicYear"], firstrow["AggregateLevel"], firstrow["CountyCode"], 
                               firstrow["DistrictCode"], firstrow["SchoolCode"], firstrow["CountyName"],
                               firstrow["DistrictName"], firstrow["SchoolName"], firstrow["CharterYN"],
                                       None,None,None,None,None,None,None,None,None,None,None,None,None,
                                       None,None,None,None,None,None,None]])
        #note that all reporting categories data is initialized as None so we can determine what was missing
        for row in rows:
            #write reporting category data to correct column
            school_data[0][9+mapping[df.iloc[row]["ReportingCategory"]]] = df.iloc[row]["ChronicAbsenteeismRate"]
        array = np.append(array,school_data,axis=0) #stack rows
    return pd.DataFrame(array[1:,:], columns=all_columns) #convert to df and return 

In [9]:
new_df2016 = colapse_df(df2016)
new_df2017 = colapse_df(df2017)
new_df2018 = colapse_df(df2018)

### Step 3
Write a function that generates a single DataFrame given multiple dataframes from different time periods of the same category. The resultant DataFrame should organize each school's data in chronological order.

I think we may be on different pages about what we were trying to produce in this step. I was thinking that it's organization would be as follows:

row 1: school 1, year 1

row 2: school 1, year 2

row 3: school 1, year 3

row 4: school 2, year 1

row 5: school 2, year 2, and so on.

I thought that there would be a different row per year, and we'd order the rows in the dataframe such that the schools data is grouped together and the rows are ordered chronologically. So I'm a little confused.

In [10]:
def merge_dfs(dfs):
    """
    Merges a list of multiple dataframes into a single dataframe that is ordered by school. 
    For example, if there are 3 years of data for school x, then these 3 rows will be immediately
    in succession. 
    Input:
    dfs: list of dataframes to be merged
    Output:
    master_df: merged dataframe that merges data from list of input dfs ordered by school
    """
    #concatenate all dataframes in dfs along axis 0
    master_df = pd.concat([df for df in dfs], axis=0, ignore_index=True)
    
    #generate zip of pairs of the format (row_index, school_code) for each row in master_df
    #sort these pairs by school code
    index_key = sorted(zip(master_df.index, master_df["SchoolCode"]), key=lambda x: x[1])
    
    #extract the indices from the zipped pairs sorted by school code
    #this will ensure that each schools data across multiple years appears in adjacent rows
    sorted_indices = [item[0] for item in index_key]
    return master_df.iloc[sorted_indices]

In [11]:
merged_df = merge_dfs([new_df2016, new_df2017, new_df2018])

### Step 3
We have some schools that are outside our target grade range (9-12), so we must drop all elementary and middle schools in our DataFrames.

In [12]:
#call read_text_file function on all public school types (2021)
df_21schools = read_text_file("./txt_files/school_type_txt_files/pubschls.txt")

In [13]:
#drop rows that do not have unique school name or code = 0
misfitSchoolCodes = []
for i, school in enumerate(df_21schools['School']):
    if school  == 'No Data' or school == 0:
        misfitSchoolCodes.append(i)
df_21schools = df_21schools.drop(misfitSchoolCodes,axis = 0)

In [40]:
def drop_little_ones(df, df_21schools):
    """
    Uses data in df_21schools to drop schools from dataset that are elementary/middle schools.
    Input:
    df: merged dataframe 
    df_21schools: dataframe from ./txt_files/school_type_txt_files/pubschls.txt with rows that
                  do not have unique school names dropped
    Output:
    df: dataframe with elementary/middle schools removed
    """
    school_codes = []
    for i in range(df_21schools.shape[0]):
        #CDS code is a code that concatenates county, district, and school code. As per the 
        #CA Department of Education's documentation, the school code is the last 7 characters
        #of this code
        school_codes.append(int(str(df_21schools["CDSCode"][df_21schools.index[i]])[-7:]))
    df_21schools["SchoolCode"] = school_codes #insert a new column "SchoolCode" 
    
    #generate zip of pairs of the format (row_index, school_code) for each row in df_21schools
    #sort these pairs by school code
    index_key = sorted(zip(df_21schools.index, df_21schools["SchoolCode"]), key=lambda x: x[1])
    sorted_indices = [item[0] for item in index_key]
    df_21schools = df_21schools.loc[sorted_indices]

    #since school codes are sorted, all rows with school code = 0 will be dropped with this 
    #block of code
    i = 0
    rows_to_drop = []
    while df_21schools["SchoolCode"][df_21schools.index[i]] == 0:
        rows_to_drop.append(df_21schools.index[i])
        i += 1
    df_21schools = df_21schools.drop(rows_to_drop,axis=0)
    
    #accumulate list of high_school_codes that are the school codes corresponding with 
    #schools that have "EILCode" = "HS"
    high_school_codes = []
    for i in range(df_21schools.shape[0]):
        if df_21schools["EILCode"][df_21schools.index[i]] == "HS":
            high_school_codes.append(df_21schools["SchoolCode"][df_21schools.index[i]])
            
    #drop all rows with school codes that are not in high_school_codes
    little_schools = []
    for i in range(df.shape[0]):
        if df["SchoolCode"][df.index[i]] not in high_school_codes:
            little_schools.append(df.index[i])
    
    df = df.drop(little_schools,axis=0)
    return df

In [42]:
final_df = drop_little_ones(merged_df, df_21schools)

In [43]:
final_df

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,CAR_RB,...,CAR_GF,CAR_GX,CAR_GZ,CAR_SE,CAR_SD,CAR_SS,CAR_SM,CAR_SF,CAR_SH,CAR_TA
5347,2016-17,S,32,10322,100057,Plumas,Plumas County Office of Education,Plumas County Community,All,None,...,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN
15366,2017-18,S,32,10322,100057,Plumas,Plumas County Office of Education,Plumas County Community,All,NaN,...,NaN,None,None,None,NaN,NaN,None,NaN,NaN,NaN
25368,2018-19,S,32,10322,100057,Plumas,Plumas County Office of Education,Plumas County Community,All,NaN,...,NaN,None,None,None,NaN,NaN,None,NaN,None,90
196,2016-17,S,1,61259,100065,Alameda,Oakland Unified,Oakland Unity High,All,19,...,16.9,None,None,24.1,7.4,15,None,NaN,None,14.7
10213,2017-18,S,1,61259,100065,Alameda,Oakland Unified,Oakland Unity High,All,31.3,...,16.6,None,None,16.7,8.3,15,None,NaN,NaN,14.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14768,2017-18,S,30,66464,6.12036e+06,Orange,Capistrano Unified,Opportunities for Learning - Capistrano,All,NaN,...,10.1,None,None,NaN,18.5,11.7,None,NaN,NaN,7.3
24772,2018-19,S,30,66464,6.12036e+06,Orange,Capistrano Unified,Opportunities for Learning - Capistrano,All,0,...,0.8,None,None,0,2.6,5,None,NaN,NaN,3.3
495,2016-17,S,5,10058,9.01074e+06,Calaveras,Calaveras County Office of Education,Oakendell Community,All,NaN,...,None,None,None,NaN,0,4.5,None,5,None,4.5
10504,2017-18,S,5,10058,9.01074e+06,Calaveras,Calaveras County Office of Education,Oakendell Community,All,NaN,...,None,None,None,NaN,7.7,0,None,0,None,5


In [ ]:
#copy final dataframe and save preprocessed data to .csv
final_df1 = final_df.copy(deep=True)
final_df1.reset_index()
final_df1.to_csv("./preprocessed_data/2016-19ChronAbsenteeism.csv", sep=",")

In [ ]:
final_df1.to_csv("./preprocessed_data/2016-19ChronAbsenteeism.csv", sep=",")